In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
from tqdm import tqdm

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)

In [2]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')
dset_labels = np.array(event_data_file['labels'])
dset_filenames = np.array(event_data_file['root_files'])
dset_eventids = np.array(event_data_file['event_ids'])

## Create FiTQun Cuts (will only affect test set)

In [3]:
# File paths for fiTQun results
fiTQun_e_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_e-.npz"
fiTQun_mu_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_mu-.npz"
fiTQun_gamma_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_gamma.npz"

# Load fiTQun results
f_e = np.load(fiTQun_e_path, allow_pickle=True)
f_mu = np.load(fiTQun_mu_path, allow_pickle=True)
f_gamma = np.load(fiTQun_gamma_path, allow_pickle=True)

In [4]:
fq_flag_e = f_e['flag']
fq_flag_mu = f_mu['flag']
fq_flag_gamma = f_gamma['flag']

fq_eventid_e = f_e['eventid']
fq_eventid_mu = f_mu['eventid']
fq_eventid_gamma = f_gamma['eventid']

fq_filename_e = f_e['filename']
fq_filename_mu = f_mu['filename']
fq_filename_gamma = f_gamma['filename']

In [5]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs'].astype(int)
train_idxs = idx_file['train_idxs'].astype(int)
val_idxs = idx_file['val_idxs'].astype(int)

In [6]:
fq_flag_dict={}
fq_failed_dict={}
for i, filename in enumerate(dset_filenames):
    key = (filename.split('/')[-1], dset_eventids[i])
    fq_flag_dict[key] = 0
    fq_failed_dict[key] = 0

for idx in test_idxs:
    key = (dset_filenames[idx].split('/')[-1], dset_eventids[idx])
    fq_flag_dict[key] = 0
    fq_failed_dict[key] = 1

In [7]:
for flags, filenames, eventids in [(fq_flag_e,fq_filename_e,fq_eventid_e), (fq_flag_mu,fq_filename_mu,fq_eventid_mu), (fq_flag_gamma,fq_filename_gamma,fq_eventid_gamma)]:
    for i, flag in tqdm(enumerate(flags)):
        key = (re.sub('_fiTQun','',filenames[i].split('/')[-1]), eventids[i]-1)
        fq_flag_dict[key] = 0 if flag[0]==0 and flag[1]==0 else 1
        fq_failed_dict[key] = 0

1118723it [00:11, 98961.51it/s] 
1097125it [00:12, 90853.38it/s]
1133003it [00:11, 94884.56it/s] 


In [8]:
failed = []
for key in fq_failed_dict.keys():
    failed.append(fq_failed_dict[key])
print(np.where(np.array(failed)==1)[0].shape)

(2169,)


In [9]:
print(len(fq_failed_dict.keys()))
print(len(fq_flag_dict.keys()))
print(dset_filenames.shape[0])

8834960
8834960
8834960


In [10]:
fq_flag = []
fq_failed = []

for i, filename in enumerate(dset_filenames):
    key = (filename.split('/')[-1], dset_eventids[i])
    fq_flag.append(fq_flag_dict[key])
    fq_failed.append(fq_failed_dict[key])
    
fq_flag = np.array(fq_flag)
fq_failed = np.array(fq_failed)

In [11]:
print(f"Failed: {np.where(fq_failed==1)[0].shape} Flagged: {np.where(fq_flag==1)[0].shape}")

Failed: (2169,) Flagged: (735557,)


## Create OD Veto Cuts

In [12]:
veto_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pm_full_tank_ODveto.h5'

In [13]:
odv_file = h5py.File(veto_path,'r')
odv_info = {}
for key in odv_file.keys():
    odv_info[key] = np.array(odv_file[key])

In [14]:
odv_info.keys()

dict_keys(['event_ids', 'labels', 'root_files', 'veto', 'veto2'])

In [15]:
odv_dict_1 = {}
odv_dict_2 = {}
pbar = ProgressBar(widgets=['Creating Event-Index Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(odv_info['event_ids']))
pbar.start()
for i, veto in enumerate(odv_info['veto']):
    odv_dict_1[(odv_info['root_files'][i], odv_info['event_ids'][i])] = veto
    odv_dict_2[(odv_info['root_files'][i], odv_info['event_ids'][i])] = odv_info['veto2'][i]
    pbar.update(i)
pbar.finish()

Creating Event-Index Dictionary: 100% [0000000000000000000000000] Time: 0:00:24


In [16]:
veto1 = []
veto2 = []
for i, filename in enumerate(dset_filenames):
    key = (filename, dset_eventids[i])
    veto1.append(odv_dict_1[key])
    veto2.append(odv_dict_2[key])

veto1 = np.array(veto1)
veto2 = np.array(veto2)

## Save file

In [17]:
np.savez(os.path.join(os.getcwd(), 'pointnet_cut_indices'), fq_flag=fq_flag, fq_failed=fq_failed, veto1=veto1, veto2=veto2)